In [1]:
import matplotlib.pyplot as plt
from modeling.evaluate import Evaluate
from modeling.predict import Predict
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import warnings
import pandas as pd
warnings.simplefilter(action="ignore")
plt.style.use("seaborn-v0_8-darkgrid")


In [2]:
df = pd.read_csv('database/DIM_PLAYER_GAMES.csv')

In [3]:
top_180_eval = [
    Evaluate(
        df=df, player=name, steps=17, regressor=RandomForestRegressor, lags=9
    ).eval()
    for name in list(set(df['PLAYER']))
]

df_top_180 = pd.concat(top_180_eval)

Not enough data. Adjusting lags for Tyjae Spears.
Not enough data. Adjusting lags for De'Von Achane.
Not enough data. Adjusting lags for Zay Flowers.
Not enough data. Adjusting steps for Isaiah Likely.
Not enough data. Adjusting steps for Desmond Ridder.
Not enough data. Adjusting lags for Zach Charbonnet.
Not enough data. Adjusting lags for Easton Stick.
Not enough data. Adjusting steps for Kyren Williams.
Not enough data. Adjusting steps for James Cook.
Not enough data. Adjusting steps for Jaylen Warren.
Not enough data. Adjusting lags for Jake Browning.
Not enough data. Adjusting lags for Bryce Young.
Not enough data. Adjusting steps for Jerome Ford.
Not enough data. Adjusting lags for Josh Downs.
Not enough data. Adjusting steps for Breece Hall.
Not enough data. Adjusting lags for Jordan Addison.
Not enough data. Adjusting lags for Jaxon Smith-Njigba.
Not enough data. Adjusting steps for Nick Mullens.
Not enough data. Adjusting steps for Kenny Pickett.
Not enough data. Adjusting la

In [4]:
np.mean(df_top_180['SEASON MAPE']), np.mean(df_top_180['GAME RSME']), np.mean(df_top_180['GAME MAPE'])

(0.2915313964992749, 8.193352464110758, 1275231635640061.8)

In [5]:
np.sqrt(mean_squared_error(df_top_180['ACTUAL SEASON TOTAL'], df_top_180['PREDICTED']))

56.690422774080844

In [6]:
top_180_pred = [
Predict(
    df=df, player=i, steps=17, regressor=GradientBoostingRegressor, lags=9
).predict_season() for i in list(set(df['PLAYER']))
]

clean_data = [row for row in top_180_pred if row is not None]

Easton Stick has insufficient data. Only 6 games on record
Jake Browning has insufficient data. Only 9 games on record
Tommy DeVito has insufficient data. Only 9 games on record
Will Levis has insufficient data. Only 9 games on record
Anthony Richardson has insufficient data. Only 4 games on record


In [7]:
df_preds = pd.DataFrame(clean_data, columns=['PLAYER', 'PROJECTED POINTS'])

In [8]:
df_2023 = df.loc[df['YEAR']==2023]
df_players_2023 = df_2023.groupby(['PLAYER', 'TEAM'])['PPR'].sum().reset_index()
df_players_2023.rename(columns={'PPR':'PPR_2023'}, inplace=True)

In [9]:
df_adp = pd.read_csv('ADP_2024.csv')
df_adp['Position'] = df_adp['Consensus Pos'].str[:2]
df_adp['Target Round'] = (df_adp['Consensus'] // 10).astype(int) + 1

In [10]:
df_merge1 = pd.merge(df_players_2023, df_preds,  how='inner', on='PLAYER')

In [11]:
df_final = pd.merge(df_merge1, df_adp[['Player', 'Position', 'Consensus', 'Target Round']], left_on='PLAYER', right_on='Player', how='inner').drop(columns=['Player'])

In [12]:
ranked_values = {
    'QB': df_final[df_final['Position'] == 'QB'].nlargest(10, 'PROJECTED POINTS')['PROJECTED POINTS'].iloc[-1] if len(df_final[df_final['Position'] == 'QB']) >= 10 else df_final[df_final['Position'] == 'QB']['PROJECTED POINTS'].min(),
    'TE': df_final[df_final['Position'] == 'TE'].nlargest(10, 'PROJECTED POINTS')['PROJECTED POINTS'].iloc[-1] if len(df_final[df_final['Position'] == 'TE']) >= 10 else df_final[df_final['Position'] == 'TE']['PROJECTED POINTS'].min(),
    'WR': df_final[df_final['Position'] == 'WR'].nlargest(30, 'PROJECTED POINTS')['PROJECTED POINTS'].iloc[-1] if len(df_final[df_final['Position'] == 'WR']) >= 30 else df_final[df_final['Position'] == 'WR']['PROJECTED POINTS'].min(),
    'RB': df_final[df_final['Position'] == 'RB'].nlargest(20, 'PROJECTED POINTS')['PROJECTED POINTS'].iloc[-1] if len(df_final[df_final['Position'] == 'RB']) >= 20 else df_final[df_final['Position'] == 'RB']['PPROJECTED POINTS'].min()
}

# Print the ranked PPR values for debugging
print("Ranked PPR values:")
print(ranked_values)

Ranked PPR values:
{'QB': 337.6109885148496, 'TE': 184.53266589377128, 'WR': 210.5260954459529, 'RB': 226.92921916285178}


In [13]:
def VBD(row):
    category = row['Position']
    ranked_ppr = ranked_values.get(category, 0) 
    return row['PROJECTED POINTS'] - ranked_ppr

# Apply the function to each row
df_final['VBD'] = df_final.apply(VBD, axis=1)


In [14]:
df_final['My Ranking'] = df_final['VBD'].rank(ascending=False, method='min').astype(int)
df_final.sort_values(by='VBD', ascending=False).head(25)

,PLAYER,TEAM,PPR_2023,PROJECTED POINTS,Position,Consensus,Target Round,VBD,My Ranking
22,CeeDee Lamb,DAL,405.20,476.543399,WR,2,1,266.017303,1
122,Nico Collins,HOU,258.40,404.129407,WR,23,3,193.603311,2
111,Kyren Williams,LAR,257.00,418.311881,RB,17,2,191.382661,3
26,Christian McCaffrey,SFO,395.30,399.726826,RB,1,1,172.797607,4
8,Amon-Ra St. Brown,DET,332.90,368.903035,WR,8,1,158.376939,5
41,Davante Adams,LVR,265.40,352.386919,WR,16,2,141.860823,6
67,Ja'Marr Chase,CIN,262.72,352.144814,WR,4,1,141.618718,7
16,Breece Hall,NYJ,288.50,363.868870,RB,5,1,136.939651,8
157,Tyreek Hill,MIA,378.40,340.178647,WR,3,1,129.652552,9
79,Jayden Reed,GNB,215.20,338.902043,WR,74,8,128.375947,10


In [16]:
df_final.to_csv('Draft Strategy 2024.csv')